<a href="https://colab.research.google.com/github/KamilBartosik/RNN_AirPolutionPrediction/blob/main/RNN_AirPolutionData.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive
